# Data Mining Project- Part I: Exploratory Data Analysis #

In this notebook, we will start the analysis of the dataset "DM2425_ABCDEats_DATASET.csv". As such, we will go through a few stages such as data processing and cleaning, data analysis and visualization.

## Table of Contents:
1. **Importing the necessary packages and the main dataframe:**
    - Importing the necessary packages such as pandas, numpy, matplotlib, and seaborn.
    - Loading the dataset.
2. **Exploring the Dataset:**
    - Summarizing essential statistics to understand the dataset's structure and the significance of each feature.
    - Identifying missing values and treating them.
    - Checking for duplicates.
    - Exploring categorical variables and numerical variables.
    - Identifying and treating anomalous values.
3. **Identifying Patterns:**
    - Detecting trends, relationships, and anomalies that may impact further analysis or decision-making.
    - Identifying correlations between features.
4. **Feature Creation:**
    - Creating new features that could enrich the dataset and potentially reveal deeper insights.
5. **Visualization:**
    - Creating visualizations to explore the data and identify potencial patterns and relationships between the features.

**Work developed by:**  
Maria Pereira 20240314  
Miguel Mangerona 20240595  
Rita Morgadito 20240611 

In [129]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns

### Initial analysis of the data set ### 

In [130]:
df=pd.read_csv("DM2425_ABCDEats_DATASET.csv")

In [131]:
df.tail()

,customer_id,customer_region,customer_age,vendor_count,product_count,is_chain,first_order,last_order,last_promo,payment_method,...,HR_14,HR_15,HR_16,HR_17,HR_18,HR_19,HR_20,HR_21,HR_22,HR_23
31883,f4e366c281,8670,30.0,1,1,1,NaN,0,FREEBIE,DIGI,...,0,0,0,0,0,0,0,0,0,0
31884,f6b6709018,8670,NaN,1,1,0,NaN,0,DELIVERY,DIGI,...,0,0,1,0,0,0,0,0,0,0
31885,f74ad8ce3f,8670,24.0,1,1,1,NaN,0,-,DIGI,...,0,0,0,0,0,0,0,0,0,0
31886,f7b19c0241,8670,34.0,1,1,0,NaN,0,DISCOUNT,DIGI,...,0,0,0,0,0,0,0,0,0,0
31887,fd40d3b0e0,4660,30.0,1,1,0,NaN,0,-,CASH,...,0,0,1,0,0,0,0,0,0,0


In [132]:
print('The dataset has', df.shape[0], 'rows and', df.shape[1], 'columns.')

The dataset has 31888 rows and 56 columns.


In [133]:
#Substitute the Nan values with the median of the respective column
df['customer_age'].fillna(df['customer_age'].median(), inplace=True)

C:\Users\migue\AppData\Local\Temp\ipykernel_14488\338681763.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['customer_age'].fillna(df['customer_age'].median(), inplace=True)


In [134]:
#Switching the customer_id and customer_region columns to integer type for numerical operations
df['customer_age'] = df['customer_age'].astype(int)

In [135]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31888 entries, 0 to 31887
Data columns (total 56 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   customer_id               31888 non-null  object 
 1   customer_region           31888 non-null  object 
 2   customer_age              31888 non-null  int32  
 3   vendor_count              31888 non-null  int64  
 4   product_count             31888 non-null  int64  
 5   is_chain                  31888 non-null  int64  
 6   first_order               31782 non-null  float64
 7   last_order                31888 non-null  int64  
 8   last_promo                31888 non-null  object 
 9   payment_method            31888 non-null  object 
 10  CUI_American              31888 non-null  float64
 11  CUI_Asian                 31888 non-null  float64
 12  CUI_Beverages             31888 non-null  float64
 13  CUI_Cafe                  31888 non-null  float64
 14  CUI_Ch

The `.describe()` is a method that provides summaru statistics of the dataframe.
 - `df.describe()`: this method only includes numeric columns
 - `df.describe(include='object')`: this method focud on object-type cokumns which usually include categorical values 
- `df.describe(include='all')`: this method show as statistics from both numerical and object columns 

In [136]:
#Checking statistics of the columns with numerical type
df.describe()

,customer_age,vendor_count,product_count,is_chain,first_order,last_order,CUI_American,CUI_Asian,CUI_Beverages,CUI_Cafe,...,HR_14,HR_15,HR_16,HR_17,HR_18,HR_19,HR_20,HR_21,HR_22,HR_23
count,31888.000000,31888.000000,31888.000000,31888.000000,31782.000000,31888.000000,31888.000000,31888.000000,31888.000000,31888.000000,...,31888.000000,31888.000000,31888.000000,31888.000000,31888.000000,31888.000000,31888.000000,31888.000000,31888.000000,31888.000000
mean,27.472153,3.102609,5.668245,2.818866,28.478604,63.675521,4.880438,9.960451,2.300633,0.801163,...,0.215630,0.277032,0.356435,0.390962,0.336961,0.245610,0.142812,0.071155,0.048263,0.045189
std,7.082367,2.771587,6.957287,3.977529,24.109086,23.226123,11.654018,23.564351,8.479734,6.427132,...,0.599006,0.738162,0.874449,0.943721,0.893949,0.795296,0.586529,0.348536,0.298265,0.282006
min,15.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,23.000000,1.000000,2.000000,1.000000,7.000000,49.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,26.000000,2.000000,3.000000,2.000000,22.000000,70.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,31.000000,4.000000,7.000000,3.000000,45.000000,83.000000,5.660000,11.830000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,80.000000,41.000000,269.000000,83.000000,90.000000,90.000000,280.210000,896.710000,229.220000,326.100000,...,13.000000,23.000000,22.000000,20.000000,24.000000,35.000000,36.000000,11.000000,15.000000,13.000000


In [137]:
#Checking statistics of the columns with object type
df.describe(include="object").T

,count,unique,top,freq
customer_id,31888,31875,742ca068fc,2
customer_region,31888,9,8670,9761
last_promo,31888,4,-,16748
payment_method,31888,3,CARD,20161


In [138]:
df.describe(include = "all")

,customer_id,customer_region,customer_age,vendor_count,product_count,is_chain,first_order,last_order,last_promo,payment_method,...,HR_14,HR_15,HR_16,HR_17,HR_18,HR_19,HR_20,HR_21,HR_22,HR_23
count,31888,31888,31888.000000,31888.000000,31888.000000,31888.000000,31782.000000,31888.000000,31888,31888,...,31888.000000,31888.000000,31888.000000,31888.000000,31888.000000,31888.000000,31888.000000,31888.000000,31888.000000,31888.000000
unique,31875,9,NaN,NaN,NaN,NaN,NaN,NaN,4,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,742ca068fc,8670,NaN,NaN,NaN,NaN,NaN,NaN,-,CARD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,2,9761,NaN,NaN,NaN,NaN,NaN,NaN,16748,20161,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,27.472153,3.102609,5.668245,2.818866,28.478604,63.675521,NaN,NaN,...,0.215630,0.277032,0.356435,0.390962,0.336961,0.245610,0.142812,0.071155,0.048263,0.045189
std,NaN,NaN,7.082367,2.771587,6.957287,3.977529,24.109086,23.226123,NaN,NaN,...,0.599006,0.738162,0.874449,0.943721,0.893949,0.795296,0.586529,0.348536,0.298265,0.282006
min,NaN,NaN,15.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,NaN,NaN,23.000000,1.000000,2.000000,1.000000,7.000000,49.000000,NaN,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,NaN,NaN,26.000000,2.000000,3.000000,2.000000,22.000000,70.000000,NaN,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,NaN,NaN,31.000000,4.000000,7.000000,3.000000,45.000000,83.000000,NaN,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


### Search for trends, patterns, or anomalies within the dataset ###


In [139]:
#Checking for missing values
df.isna().sum()

customer_id                    0
customer_region                0
customer_age                   0
vendor_count                   0
product_count                  0
is_chain                       0
first_order                  106
last_order                     0
last_promo                     0
payment_method                 0
CUI_American                   0
CUI_Asian                      0
CUI_Beverages                  0
CUI_Cafe                       0
CUI_Chicken Dishes             0
CUI_Chinese                    0
CUI_Desserts                   0
CUI_Healthy                    0
CUI_Indian                     0
CUI_Italian                    0
CUI_Japanese                   0
CUI_Noodle Dishes              0
CUI_OTHER                      0
CUI_Street Food / Snacks       0
CUI_Thai                       0
DOW_0                          0
DOW_1                          0
DOW_2                          0
DOW_3                          0
DOW_4                          0
DOW_5     

The `.isna()` gives as *False* if the value exist and *True* if the value in the cell is missing 

In [140]:
df.isna()

,customer_id,customer_region,customer_age,vendor_count,product_count,is_chain,first_order,last_order,last_promo,payment_method,...,HR_14,HR_15,HR_16,HR_17,HR_18,HR_19,HR_20,HR_21,HR_22,HR_23
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31883,False,False,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
31884,False,False,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
31885,False,False,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
31886,False,False,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [141]:
#Visualizing which regions exist
df['customer_region'].unique()

array(['2360', '8670', '4660', '2440', '-', '4140', '2490', '8370',
       '8550'], dtype=object)

In [142]:
df['payment_method'].unique()

array(['DIGI', 'CASH', 'CARD'], dtype=object)

In [143]:
#Replacing empty customer regions with Nan
df["customer_region"]=df["customer_region"].replace("-",np.nan)

In [144]:
#Checking if there are any duplicated values in the column customer id
print(df['customer_id'].duplicated().sum())

13


In [145]:
#Concatenate those columns
pd.concat(identifier for _, identifier in df.groupby('customer_id') if len(identifier) > 1)

,customer_id,customer_region,customer_age,vendor_count,product_count,is_chain,first_order,last_order,last_promo,payment_method,...,HR_14,HR_15,HR_16,HR_17,HR_18,HR_19,HR_20,HR_21,HR_22,HR_23
28706,06018a56be,8550,27,2,2,0,67.0,89,FREEBIE,DIGI,...,0,0,0,0,0,0,0,0,0,0
28713,06018a56be,8550,27,2,2,0,67.0,89,FREEBIE,DIGI,...,0,0,0,0,0,0,0,0,0,0
27653,201a13a34d,8550,28,2,2,1,62.0,62,DELIVERY,CARD,...,0,0,0,0,0,0,0,0,0,1
27739,201a13a34d,8550,28,2,2,1,62.0,62,DELIVERY,CARD,...,0,0,0,0,0,0,0,0,0,1
7867,24251eb7da,8550,28,4,8,3,7.0,85,-,CARD,...,0,1,0,1,0,0,0,0,0,0
7887,24251eb7da,8550,28,4,8,3,7.0,85,-,CARD,...,0,1,0,1,0,0,0,0,0,0
16227,671bf0c738,8550,25,4,7,3,23.0,66,FREEBIE,CARD,...,0,0,0,1,0,1,0,0,0,0
16230,671bf0c738,8550,25,4,7,3,23.0,66,FREEBIE,CARD,...,0,0,0,1,0,1,0,0,0,0
24822,6bbf5f74cd,8550,22,10,20,9,48.0,88,-,CARD,...,0,0,0,0,0,0,0,0,0,0
24827,6bbf5f74cd,8550,22,10,20,9,48.0,88,-,CARD,...,0,0,0,0,0,0,0,0,0,0


In [146]:
#Showing which are the not numerical columns, and their corresponding values (unique values only)
non_numerical_columns = df.select_dtypes(exclude=['number']).columns

for column in non_numerical_columns:
    unique_values = df[column].unique()
    print(f"Column name: {column}")
    print(f"Corresponding values: {unique_values}\n")


Column name: customer_id
Corresponding values: ['1b8f824d5e' '5d272b9dcb' 'f6d1b2ba63' ... 'f74ad8ce3f' 'f7b19c0241'
 'fd40d3b0e0']

Column name: customer_region
Corresponding values: ['2360' '8670' '4660' '2440' nan '4140' '2490' '8370' '8550']

Column name: last_promo
Corresponding values: ['DELIVERY' 'DISCOUNT' '-' 'FREEBIE']

Column name: payment_method
Corresponding values: ['DIGI' 'CASH' 'CARD']



In [147]:
#Showing the duplicated rows
df[df.duplicated()==True]

,customer_id,customer_region,customer_age,vendor_count,product_count,is_chain,first_order,last_order,last_promo,payment_method,...,HR_14,HR_15,HR_16,HR_17,HR_18,HR_19,HR_20,HR_21,HR_22,HR_23
3379,b55012ee1c,8550,23,4,11,1,2.0,89,FREEBIE,CARD,...,0,0,0,0,0,0,0,0,0,0
7887,24251eb7da,8550,28,4,8,3,7.0,85,-,CARD,...,0,1,0,1,0,0,0,0,0,0
16230,671bf0c738,8550,25,4,7,3,23.0,66,FREEBIE,CARD,...,0,0,0,1,0,1,0,0,0,0
20907,742ca068fc,8550,20,1,2,0,35.0,35,-,CARD,...,0,0,0,0,0,0,0,0,0,0
24329,df91183978,8550,23,2,3,1,47.0,52,DELIVERY,CARD,...,0,0,0,0,0,0,0,0,0,0
24827,6bbf5f74cd,8550,22,10,20,9,48.0,88,-,CARD,...,0,0,0,0,0,0,0,0,0,0
25793,8aa9bbc147,8550,24,2,4,2,53.0,59,DELIVERY,DIGI,...,0,0,0,0,0,0,0,0,0,0
27515,cf563a0a98,8550,27,2,2,0,61.0,62,FREEBIE,CASH,...,0,0,0,0,0,0,0,0,0,0
27739,201a13a34d,8550,28,2,2,1,62.0,62,DELIVERY,CARD,...,0,0,0,0,0,0,0,0,0,1
28713,06018a56be,8550,27,2,2,0,67.0,89,FREEBIE,DIGI,...,0,0,0,0,0,0,0,0,0,0


In [148]:
df.duplicated().sum() #testes

13

In [149]:
df['customer_id'].duplicated().sum() #testes

13

In [150]:
df[df['customer_id'].duplicated()].head(13)

,customer_id,customer_region,customer_age,vendor_count,product_count,is_chain,first_order,last_order,last_promo,payment_method,...,HR_14,HR_15,HR_16,HR_17,HR_18,HR_19,HR_20,HR_21,HR_22,HR_23
3379,b55012ee1c,8550,23,4,11,1,2.0,89,FREEBIE,CARD,...,0,0,0,0,0,0,0,0,0,0
7887,24251eb7da,8550,28,4,8,3,7.0,85,-,CARD,...,0,1,0,1,0,0,0,0,0,0
16230,671bf0c738,8550,25,4,7,3,23.0,66,FREEBIE,CARD,...,0,0,0,1,0,1,0,0,0,0
20907,742ca068fc,8550,20,1,2,0,35.0,35,-,CARD,...,0,0,0,0,0,0,0,0,0,0
24329,df91183978,8550,23,2,3,1,47.0,52,DELIVERY,CARD,...,0,0,0,0,0,0,0,0,0,0
24827,6bbf5f74cd,8550,22,10,20,9,48.0,88,-,CARD,...,0,0,0,0,0,0,0,0,0,0
25793,8aa9bbc147,8550,24,2,4,2,53.0,59,DELIVERY,DIGI,...,0,0,0,0,0,0,0,0,0,0
27515,cf563a0a98,8550,27,2,2,0,61.0,62,FREEBIE,CASH,...,0,0,0,0,0,0,0,0,0,0
27739,201a13a34d,8550,28,2,2,1,62.0,62,DELIVERY,CARD,...,0,0,0,0,0,0,0,0,0,1
28713,06018a56be,8550,27,2,2,0,67.0,89,FREEBIE,DIGI,...,0,0,0,0,0,0,0,0,0,0


In [151]:
df[df['customer_id'].duplicated(keep='last')].head(13)

,customer_id,customer_region,customer_age,vendor_count,product_count,is_chain,first_order,last_order,last_promo,payment_method,...,HR_14,HR_15,HR_16,HR_17,HR_18,HR_19,HR_20,HR_21,HR_22,HR_23
3355,b55012ee1c,8550,23,4,11,1,2.0,89,FREEBIE,CARD,...,0,0,0,0,0,0,0,0,0,0
7867,24251eb7da,8550,28,4,8,3,7.0,85,-,CARD,...,0,1,0,1,0,0,0,0,0,0
16227,671bf0c738,8550,25,4,7,3,23.0,66,FREEBIE,CARD,...,0,0,0,1,0,1,0,0,0,0
20855,742ca068fc,8550,20,1,2,0,35.0,35,-,CARD,...,0,0,0,0,0,0,0,0,0,0
24328,df91183978,8550,23,2,3,1,47.0,52,DELIVERY,CARD,...,0,0,0,0,0,0,0,0,0,0
24822,6bbf5f74cd,8550,22,10,20,9,48.0,88,-,CARD,...,0,0,0,0,0,0,0,0,0,0
25792,8aa9bbc147,8550,24,2,4,2,53.0,59,DELIVERY,DIGI,...,0,0,0,0,0,0,0,0,0,0
27514,cf563a0a98,8550,27,2,2,0,61.0,62,FREEBIE,CASH,...,0,0,0,0,0,0,0,0,0,0
27653,201a13a34d,8550,28,2,2,1,62.0,62,DELIVERY,CARD,...,0,0,0,0,0,0,0,0,0,1
28706,06018a56be,8550,27,2,2,0,67.0,89,FREEBIE,DIGI,...,0,0,0,0,0,0,0,0,0,0


In [152]:
#testes maria
df.shape

(31888, 56)

In [153]:
df['customer_id'].drop_duplicates().shape

(31875,)

In [154]:
df.drop_duplicates(subset = ['customer_id','customer_region']).shape

(31875, 56)

In [155]:
df.drop_duplicates(subset = ['customer_id']).shape

(31875, 56)

In [156]:
df.drop_duplicates(subset = ['customer_region']).shape

(9, 56)

In [157]:
#Antes de fazer os gráficos devíamos adicionar colunas relevantes como eles têm

In [158]:
#Setting the customer id as the index of the df
df.set_index('customer_id', inplace=True)

### Creating new features ###

#### Money Spent ####

In [159]:
cui_columns = [col for col in df.columns if col.startswith('CUI')]
df['tot_value_cui'] = df[cui_columns].sum(axis=1)

#### Order Duration ####

In [160]:
#Creating a column that stores the calculation of days since the first order
df['order_duration'] = df['last_order'] - df['first_order']

#### Order Frequency ####

In [161]:
#Creating a column that stores the frequency of how often a customer places orders (based on their first_order and last_order values)
df['order_frequency'] = df['product_count'] / (df['last_order'] - df['first_order'] + 1) 

#### Prefered time of the day ####

In [162]:
df['preferred_order_hour'] = df[[f'HR_{i}' for i in range(24)]].idxmax(axis=1).str.replace('HR_', '').astype(int)

#### Prefered day of the week ####

In [163]:
df['preferred_order_day'] = df[[f'DOW_{i}' for i in range(7)]].idxmax(axis=1).str.replace('DOW_', '').astype(int)

#### Age group ####

In [164]:
#Creating a column that stores the age group to which the consumer belongs
bins = [0, 18, 25, 35, 45, 55, 65, 100]
labels = ['0-17', '18-24', '25-34', '35-44', '45-54', '55-64', '65+']

# Create the age_category column
df['age_category'] = pd.cut(df['customer_age'], bins=bins, labels=labels, right=False)

### Graphs ###

In [ ]:

hourly_columns = [f'HR_{i}' for i in range(24)]

hourly_totals = df[hourly_columns].sum()

plt.figure(figsize=(12, 6))
sns.barplot(x=hourly_totals.index.str.replace("HR_", ""), y=hourly_totals.values, color="orange")

plt.title('Total Number of Orders by Hour of the Day')
plt.xlabel('Hour of the Day')
plt.ylabel('Total Orders')
plt.xticks(rotation=45)
plt.show()


In [ ]:
weekly_columns = [f'DOW_{i}' for i in range(7)]

weekly_totals = df[weekly_columns].sum()

plt.figure(figsize=(10, 6))
sns.barplot(x=weekly_totals.index.str.replace("DOW_", ""), y=weekly_totals.values, color="gold")

plt.title('Total Number of Orders by Day of the Week')
plt.xlabel('Day of the Week (0 = Sunday, 6 = Saturday)')
plt.ylabel('Total Orders')
plt.show()


**Feature: age**

In [ ]:
df['customer_age'].describe()

In [ ]:

#ESTOU A INVENTAR AMIGUINHOS

# Verificar duplicatas no índice
if df.index.has_duplicates:
    df = df.reset_index(drop=True)  # Redefine o índice, removendo duplicatas


plt.figure(figsize=(8, 6))
sns.boxplot(x=df['customer_age'], color='lightgreen')
plt.title('Box Plot de Idade')
plt.xlabel('Idade')
plt.show()